In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/webis-clickbait-22/test.jsonl
/kaggle/input/webis-clickbait-22/validation.jsonl
/kaggle/input/webis-clickbait-22/train.jsonl


In [2]:
! pip install accelarate -U
! pip install datasets

ERROR: Could not find a version that satisfies the requirement accelarate (from versions: none)
ERROR: No matching distribution found for accelarate


In [3]:
import wandb
api_key = "9963cf6219e451d47251ea34645181ada1b2526b"
wandb.login(key=api_key)
wandb.init()

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: raghav21274 (ragha). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.16.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.5
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240422_111920-9dodut02
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run hearty-lake-66
wandb: ⭐️ View project at https://wandb.ai/ragha/uncategorized
wandb: 🚀 View run at https://wandb.ai/ragha/uncategorized/runs/9dodut02/workspace


In [4]:
import torch
from datasets import load_dataset
from transformers import (
    DebertaTokenizer,
    DebertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
)
# from sklearnex import patch_sklearn
# patch_sklearn()

import json
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pickle
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report


PATH="/kaggle/input/webis-clickbait-22/"
sep='[SEP]'

label_encoder = LabelEncoder()
model_id = "microsoft/deberta-base"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

2024-04-22 11:19:46.457624: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-22 11:19:46.457721: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-22 11:19:46.588079: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
def get_split_data(file_path,if_train=True):
    global label_encoder
    data=pd.read_json(file_path,lines=True,encoding='utf-8')

    y = data["tags"]
    Y = [i[0] for i in y]
    print("len of Y: ",len(Y))
    if if_train:
        Y = label_encoder.fit_transform(Y)
    else:
        Y = label_encoder.transform(Y)

    new_data=data[['postText','targetParagraphs','targetTitle','targetDescription','targetKeywords',"targetMedia"]]
    dic=new_data.to_dict(orient='records')

    desccount=0
    keycount=0
    media_count=0
    for i in dic:
        # print(i)
        i['postText']='postText: '+i['postText'][0]
        targetparah=" ".join(i['targetParagraphs'])
        i['targetParagraphs']='targetParagraphs: '+targetparah
        i['targetTitle']='targetTitle: '+i['targetTitle']
        targetDescription=i['targetDescription']
        if(targetDescription is None or len(targetDescription)==0):
            targetDescription='none'
            desccount+=1
        i['targetDescription']='targetDescription: '+targetDescription
        targetkey=i['targetKeywords']
        if(targetkey is None or len(targetkey)==0):
            targetkey='none'
            keycount+=1
        i['targetKeywords']='targetKeywords: '+targetkey
        i['text']=i['targetParagraphs']+sep+i['postText']
        del i['postText']
        del i['targetParagraphs']
        del i['targetTitle']
        del i['targetDescription']
        del i['targetKeywords']

    print("desccount: ",desccount)
    print("keycount: ",keycount)
    X = [i['text'] for i in dic]
    X=np.array(X)
#     return X[0:100],Y[0:100]
    return X,Y

In [6]:
x_train,y_train = get_split_data(PATH+'train.jsonl')
x_test,y_test = get_split_data(PATH+'test.jsonl',False)
x_val,y_val = get_split_data(PATH+'validation.jsonl',False)

len of Y:  3200
desccount:  332
keycount:  1314
len of Y:  1000
desccount:  104
keycount:  414
len of Y:  800
desccount:  78
keycount:  330


In [7]:
x_train[0]

'targetParagraphs: It’ll be just like old times this weekend for Tom Brady and Wes Welker. Welker revealed Friday morning on a Miami radio station that he contacted Brady because he’ll be in town for Sunday’s game between the New England Patriots and Miami Dolphins at Gillette Stadium. It seemed like a perfect opportunity for the two to catch up. But Brady’s definition of "catching up" involves far more than just a meal. In fact, it involves some literal "catching" as the Patriots quarterback looks to stay sharp during his four-game Deflategate suspension. "I hit him up to do dinner Saturday night. He’s like, ‘I’m going to be flying in from Ann Arbor later (after the Michigan-Colorado football game), but how about that morning we go throw?’ " Welker said on WQAM, per The Boston Globe. "And I’m just sitting there, I’m like, ‘I was just thinking about dinner, but yeah, sure. I’ll get over there early and we can throw a little bit.’ " Welker was one of Brady’s favorite targets for six sea

In [8]:
num=len(label_encoder.classes_)
id_to_label = {int(idd):label for label, idd in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))}
print("ID to Label mapping:", id_to_label)
label_to_id  = {label:int(idd) for label, idd in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))}
print("Label to ID mapping:", label_to_id)

ID to Label mapping: {0: 'multi', 1: 'passage', 2: 'phrase'}
Label to ID mapping: {'multi': 0, 'passage': 1, 'phrase': 2}


In [9]:
config =AutoConfig.from_pretrained(model_id)
config.update({"id2label": id_to_label})
config.update({"label2id":label_to_id})
config.update({"num_labels":3})
tokenizer = DebertaTokenizer.from_pretrained(model_id)
model = DebertaForSequenceClassification.from_pretrained(model_id,config=config)

config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/559M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
model = model.to(device)
model

DebertaForSequenceClassification(
  (deberta): DebertaModel(
    (embeddings): DebertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=0)
      (LayerNorm): DebertaLayerNorm()
      (dropout): StableDropout()
    )
    (encoder): DebertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaLayer(
          (attention): DebertaAttention(
            (self): DisentangledSelfAttention(
              (in_proj): Linear(in_features=768, out_features=2304, bias=False)
              (pos_dropout): StableDropout()
              (pos_proj): Linear(in_features=768, out_features=768, bias=False)
              (pos_q_proj): Linear(in_features=768, out_features=768, bias=True)
              (dropout): StableDropout()
            )
            (output): DebertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): DebertaLayerNorm()
              (dropout): StableDropout()
            )
          )
          (

In [11]:
x_train_tokenized = tokenizer(x_train.tolist(),padding=True,truncation=True,max_length=512)
x_val_tokenized = tokenizer(x_val.tolist(),padding=True,truncation=True,max_length=512)
x_test_tokenized = tokenizer(x_test.tolist(),padding=True,truncation=True,max_length=512)

In [12]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self,encodings,labels=None):
        self.labels = labels
        self.encodings = encodings
    
    def __getitem__(self,idx):
        item = {key:torch.tensor(val[idx]) for key,val in self.encodings.items()}
        if self.labels is not None:
            item["labels"]  = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])
            

In [13]:
train_dataset = Dataset(x_train_tokenized,y_train)
val_dataset = Dataset(x_val_tokenized,y_val)
test_dataset = Dataset(x_test_tokenized,y_test)

In [14]:
train_dataset[0]

{'input_ids': tensor([    1, 23976, 22011, 44947,    29,    35,    85,    17,    27,   890,
            28,    95,   101,   793,   498,    42,   983,    13,  1560,  5333,
             8, 15590, 15565,  5029,     4, 15565,  5029,  1487,   273,   662,
            15,    10,  2561,  3188,  1992,    14,    37,  5590,  5333,   142,
            37,    17,    27,   890,    28,    11,  1139,    13,   395,    17,
            27,    29,   177,   227,     5,   188,  1156,  4314,     8,  2561,
          8764,    23,  8205, 13476,  2689,     4,    85,  2551,   101,    10,
          1969,   945,    13,     5,    80,     7,  2916,    62,     4,   125,
          5333,    17,    27,    29,  8515,     9,    22, 24882,    62,   113,
          6890,   444,    55,    87,    95,    10,  5820,     4,    96,   754,
             6,    24,  6890,   103, 27856,    22, 24882,   113,    25,     5,
          4314,  2525,  1326,     7,  1095,  4406,   148,    39,   237,    12,
          2670,  7858,   462, 27586,   

In [15]:
from transformers import TrainerCallback
import torch

def compute_metrics(eval_preds):
    preds,labels = eval_preds
    preds = np.argmax(preds,axis=1)
    acc = balanced_accuracy_score(labels,preds)
    f1 = f1_score(labels,preds,average="macro")
    return {"balanced_accuracy":acc,"f1_score":f1}
    
class LoggingCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        print(f"Training Loss: {state.log_history['loss'][-1]}")

In [16]:
args = TrainingArguments(
    output_dir = "/kaggle/working/",
    num_train_epochs=10,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=14,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=500,
    load_best_model_at_end=True,
)
trainer = Trainer(
    model = model,
    args = args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    compute_metrics =compute_metrics,
#     callbacks=[LoggingCallback()]
    
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [17]:
trainer.train()

Epoch,Training Loss,Validation Loss,Balanced Accuracy,F1 Score
1,No log,0.908352,0.523885,0.536841
2,0.987200,0.871809,0.592310,0.610731
3,0.831200,0.851452,0.618336,0.636365
4,0.585900,0.963681,0.644622,0.661315
5,0.361300,1.411916,0.627818,0.643095
6,0.361300,1.731855,0.631060,0.640467
7,0.202500,2.230656,0.607507,0.619599
8,0.099000,2.502412,0.609096,0.624310
9,0.054500,2.632683,0.612824,0.628037
10,0.023900,2.696275,0.617633,0.625510


TrainOutput(global_step=4000, training_loss=0.3931723668575287, metrics={'train_runtime': 3078.7197, 'train_samples_per_second': 10.394, 'train_steps_per_second': 1.299, 'total_flos': 9811198771200000.0, 'train_loss': 0.3931723668575287, 'epoch': 10.0})

In [18]:
trainer.save_model("deberta_overall")
tokenizer.save_pretrained("deberta_overall")

('deberta_overall/tokenizer_config.json',
 'deberta_overall/special_tokens_map.json',
 'deberta_overall/vocab.json',
 'deberta_overall/merges.txt',
 'deberta_overall/added_tokens.json')

In [19]:
# test model
# y_pred=model.predict(test_dataset)

from transformers import pipeline
# Set the model to evaluation mode
classifier = pipeline('text-classification',"/kaggle/working/deberta_overall",truncation=True)

y_pred=[]
for i in x_test:
    text = i
    result = classifier(text)
    predicted_label = result[0]["label"]
    y_pred.append(predicted_label)

In [20]:
y_pred=label_encoder.transform(y_pred)

In [21]:
print("Testing data metrics:")
print(balanced_accuracy_score(y_test,y_pred))
print(f1_score(y_test,y_pred,  average='macro'))
print(classification_report(y_test,y_pred))

Testing data metrics:
0.629988544787494
0.6474463303107804
              precision    recall  f1-score   support

           0       0.82      0.46      0.59       174
           1       0.60      0.85      0.71       403
           2       0.73      0.58      0.65       423

    accuracy                           0.67      1000
   macro avg       0.72      0.63      0.65      1000
weighted avg       0.70      0.67      0.66      1000

